# Interfacing Between Pandas and Model Code

Acommon workflow of the model development is the use of pandas for data loading and cleaning before swith to modeling libraries.An important part of the model building process is called feature engineering in machine learning.this can describe data transformation and analytic that extract information from row dataset that may be useful in the modelling context.

In [2]:
import pandas as pd 
import numpy as np
data=pd.DataFrame({'x0':[1,2,3,4,5],'x1':[0.01,-0.01,0.25,-4.1,0.],'y':[-1.5,0,3.6,1.3,-2.]})
data

,x0,x1,y
0,1,0.01,-1.5
1,2,-0.01,0.0
2,3,0.25,3.6
3,4,-4.10,1.3
4,5,0.00,-2.0


In [3]:
data.columns

Index(['x0', 'x1', 'y'], dtype='object')

In [4]:
data.values

array([[ 1.  ,  0.01, -1.5 ],
       [ 2.  , -0.01,  0.  ],
       [ 3.  ,  0.25,  3.6 ],
       [ 4.  , -4.1 ,  1.3 ],
       [ 5.  ,  0.  , -2.  ]])

In [5]:
#to convert back to data frame
df2=pd.DataFrame(data.values,columns=['one','two','three'])
df2

,one,two,three
0,1.0,0.01,-1.5
1,2.0,-0.01,0.0
2,3.0,0.25,3.6
3,4.0,-4.10,1.3
4,5.0,0.00,-2.0


In [6]:
df3=data.copy()

In [7]:
df3

,x0,x1,y
0,1,0.01,-1.5
1,2,-0.01,0.0
2,3,0.25,3.6
3,4,-4.10,1.3
4,5,0.00,-2.0


In [8]:
df3['strings'] = ['a', 'b', 'c', 'd', 'e']

In [9]:
df3

,x0,x1,y,strings
0,1,0.01,-1.5,a
1,2,-0.01,0.0,b
2,3,0.25,3.6,c
3,4,-4.10,1.3,d
4,5,0.00,-2.0,e


In [10]:
#.value attribute is intended to use when your data is homogenious,for example numeric data.if you have the heterogenious data,the result will be ndarray of the python object.
df3.values

array([[1, 0.01, -1.5, 'a'],
       [2, -0.01, 0.0, 'b'],
       [3, 0.25, 3.6, 'c'],
       [4, -4.1, 1.3, 'd'],
       [5, 0.0, -2.0, 'e']], dtype=object)

In [11]:
model_col=['x1','y']
data.loc[:,model_col]

,x1,y
0,0.01,-1.5
1,-0.01,0.0
2,0.25,3.6
3,-4.10,1.3
4,0.00,-2.0


In [12]:
data.loc[:,model_col].values

array([[ 0.01, -1.5 ],
       [-0.01,  0.  ],
       [ 0.25,  3.6 ],
       [-4.1 ,  1.3 ],
       [ 0.  , -2.  ]])

In [13]:
data['category']=pd.Categorical(['a','b','a','a','b'])

In [14]:
data

,x0,x1,y,category
0,1,0.01,-1.5,a
1,2,-0.01,0.0,b
2,3,0.25,3.6,a
3,4,-4.10,1.3,a
4,5,0.00,-2.0,b


In [15]:
data.category.cat.categories

Index(['a', 'b'], dtype='object')

In [16]:
data.category.cat.codes

0    0
1    1
2    0
3    0
4    1
dtype: int8

In [17]:
dummies=pd.get_dummies(data.category,prefix='category')
data_with_dummies=data.drop('category',axis=1).join(dummies)
data_with_dummies

,x0,x1,y,category_a,category_b
0,1,0.01,-1.5,1,0
1,2,-0.01,0.0,0,1
2,3,0.25,3.6,1,0
3,4,-4.10,1.3,1,0
4,5,0.00,-2.0,0,1


# Creating model description with patsy

    patsy is a python librar for describing statistic model(especialy linear model) with a small string based "formula syntex" which is inspired by(but exactly not same as) formula syntex used by the R and S statistical language.
    Patsys is well supported for specifying linear model in statmodel so i will focus on the some of main feature to help you get up and runnins.Patsy formula are a special string type syntex that look like
    y~x0+x1
    the syntex a+b does mean than addition of a and b that rather that these are term of the design matrix created for the model

In [18]:
import patsy

In [21]:
datam=pd.DataFrame({'x0':[1,2,3,4,5],'x1':[0.01,-0.01,0.25,-4.10,0.0],'y':[-1.5,0.0,3.6,1.3,-2.0]})
datam
                

,x0,x1,y
0,1,0.01,-1.5
1,2,-0.01,0.0
2,3,0.25,3.6
3,4,-4.10,1.3
4,5,0.00,-2.0


In [23]:
y,X=patsy.dmatrices('y ~ x0 + x1',data)

In [28]:
patsy.dmatrices('y ~ x0 + x1',data)[1]

DesignMatrix with shape (5, 3)
  Intercept  x0     x1
          1   1   0.01
          1   2  -0.01
          1   3   0.25
          1   4  -4.10
          1   5   0.00
  Terms:
    'Intercept' (column 0)
    'x0' (column 1)
    'x1' (column 2)

In [24]:
y

DesignMatrix with shape (5, 1)
     y
  -1.5
   0.0
   3.6
   1.3
  -2.0
  Terms:
    'y' (column 0)

In [25]:
X

DesignMatrix with shape (5, 3)
  Intercept  x0     x1
          1   1   0.01
          1   2  -0.01
          1   3   0.25
          1   4  -4.10
          1   5   0.00
  Terms:
    'Intercept' (column 0)
    'x0' (column 1)
    'x1' (column 2)

In [26]:
np.asarray(y)

array([[-1.5],
       [ 0. ],
       [ 3.6],
       [ 1.3],
       [-2. ]])

In [27]:
np.asarray(X)

array([[ 1.  ,  1.  ,  0.01],
       [ 1.  ,  2.  , -0.01],
       [ 1.  ,  3.  ,  0.25],
       [ 1.  ,  4.  , -4.1 ],
       [ 1.  ,  5.  ,  0.  ]])

In [31]:
#you may wonder from where the intercept term come from.This is convention of linear model like ordinary least squre(OLS) method.you can supress the intercept by adding the term 0 to the model.
patsy.dmatrices('y~ x0 + x1 + 0',data)[1]

DesignMatrix with shape (5, 2)
  x0     x1
   1   0.01
   2  -0.01
   3   0.25
   4  -4.10
   5   0.00
  Terms:
    'x0' (column 0)
    'x1' (column 1)

In [32]:
#patsy can be directly passed in the algorithm like numpy.linalg.lstsqq(X,y),which perform an ordinary least squre regression
coeff, resid, _, _=np.linalg.lstsq(X,y)

C:\Users\Shubhamay\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  


In [33]:
coeff

array([[ 0.31290976],
       [-0.07910564],
       [-0.26546384]])

In [35]:
#Squeeze 1 dimentional axis object into scaler.
#Series or DataFrame with a single element are squeeze into a scaler.DataFrame with a single column or single row are squeeze to a series
coeff.squeeze()

array([ 0.31290976, -0.07910564, -0.26546384])

In [36]:
type(coeff)

patsy.design_info.DesignMatrix

In [42]:
coef=pd.Series(coeff.squeeze(),index=X.design_info.column_names)
coef

Intercept    0.312910
x0          -0.079106
x1          -0.265464
dtype: float64

In [44]:
np.abs(-0.01)

0.01

In [45]:
0.01+1

1.01

In [46]:
np.log(1.01)

0.009950330853168092

# Data Transformation in Patsy formulas

In [47]:
y,X=patsy.dmatrices('y~ x0 + np.log(np.abs(x1)+1)',data)
y

DesignMatrix with shape (5, 1)
     y
  -1.5
   0.0
   3.6
   1.3
  -2.0
  Terms:
    'y' (column 0)

In [48]:
X

DesignMatrix with shape (5, 3)
  Intercept  x0  np.log(np.abs(x1) + 1)
          1   1                 0.00995
          1   2                 0.00995
          1   3                 0.22314
          1   4                 1.62924
          1   5                 0.00000
  Terms:
    'Intercept' (column 0)
    'x0' (column 1)
    'np.log(np.abs(x1) + 1)' (column 2)

In [57]:
y,X=patsy.dmatrices('y~ standardize(x0) + center(x1)',data)
y

DesignMatrix with shape (5, 1)
     y
  -1.5
   0.0
   3.6
   1.3
  -2.0
  Terms:
    'y' (column 0)

In [58]:
X

DesignMatrix with shape (5, 3)
  Intercept  standardize(x0)  center(x1)
          1         -1.41421        0.78
          1         -0.70711        0.76
          1          0.00000        1.02
          1          0.70711       -3.33
          1          1.41421        0.77
  Terms:
    'Intercept' (column 0)
    'standardize(x0)' (column 1)
    'center(x1)' (column 2)

In [62]:
new_data=pd.DataFrame({'x0':[6,7,8,9],'x1':[3.1,-0.5,0,2.3],'y':[1,2,3,4]})
new_data

,x0,x1,y
0,6,3.1,1
1,7,-0.5,2
2,8,0.0,3
3,9,2.3,4


In [64]:
new_X=patsy.build_design_matrices([X.design_info],new_data)
new_X

[DesignMatrix with shape (4, 3)
   Intercept  standardize(x0)  center(x1)
           1          2.12132        3.87
           1          2.82843        0.27
           1          3.53553        0.77
           1          4.24264        3.07
   Terms:
     'Intercept' (column 0)
     'standardize(x0)' (column 1)
     'center(x1)' (column 2)]

In [65]:
m,N=patsy.dmatrices('y~ x0 + x1 ',data)
N

DesignMatrix with shape (5, 3)
  Intercept  x0     x1
          1   1   0.01
          1   2  -0.01
          1   3   0.25
          1   4  -4.10
          1   5   0.00
  Terms:
    'Intercept' (column 0)
    'x0' (column 1)
    'x1' (column 2)

In [68]:
new_N=patsy.build_design_matrices([N.design_info],new_data)
new_N

[DesignMatrix with shape (4, 3)
   Intercept  x0    x1
           1   6   3.1
           1   7  -0.5
           1   8   0.0
           1   9   2.3
   Terms:
     'Intercept' (column 0)
     'x0' (column 1)
     'x1' (column 2)]

In [71]:
#Beacause the plus symbol(+) in the context of Patsy Formula does not mean addition,when you want to add columns dataset by name,then you must wrap them in the special I function
y,X=patsy.dmatrices('y~I(x0+x1)',data)
X

DesignMatrix with shape (5, 2)
  Intercept  I(x0 + x1)
          1        1.01
          1        1.99
          1        3.25
          1       -0.10
          1        5.00
  Terms:
    'Intercept' (column 0)
    'I(x0 + x1)' (column 1)

# Categorical Data and Patsy

In [2]:
import numpy as np
import pandas as pd
dt=pd.DataFrame({'key1':['m','m','n','n','m','n','m','n'],'key2':[0,1,0,1,0,1,0,0],'v1':[1,2,3,4,5,6,7,8],'v2':[-1,0,2.5,-0.5,4.0,-1.2,0.2,-1.7]})
dt

,key1,key2,v1,v2
0,m,0,1,-1.0
1,m,1,2,0.0
2,n,0,3,2.5
3,n,1,4,-0.5
4,m,0,5,4.0
5,n,1,6,-1.2
6,m,0,7,0.2
7,n,0,8,-1.7


In [3]:
import patsy
y,X=patsy.dmatrices('v2~key1',dt)
y

DesignMatrix with shape (8, 1)
    v2
  -1.0
   0.0
   2.5
  -0.5
   4.0
  -1.2
   0.2
  -1.7
  Terms:
    'v2' (column 0)

In [4]:
#the T notation is there to remind you that these column are treetment code
X

DesignMatrix with shape (8, 2)
  Intercept  key1[T.n]
          1          0
          1          0
          1          1
          1          1
          1          0
          1          1
          1          0
          1          1
  Terms:
    'Intercept' (column 0)
    'key1' (column 1)

In [5]:
y,X=patsy.dmatrices('v2~ 0+key1',dt)

In [6]:
y

DesignMatrix with shape (8, 1)
    v2
  -1.0
   0.0
   2.5
  -0.5
   4.0
  -1.2
   0.2
  -1.7
  Terms:
    'v2' (column 0)

In [7]:
X

DesignMatrix with shape (8, 2)
  key1[m]  key1[n]
        1        0
        1        0
        0        1
        0        1
        1        0
        0        1
        1        0
        0        1
  Terms:
    'key1' (columns 0:2)

In [8]:
patsy.dmatrices('v2~key2',dt)[1]

DesignMatrix with shape (8, 2)
  Intercept  key2
          1     0
          1     1
          1     0
          1     1
          1     0
          1     1
          1     0
          1     0
  Terms:
    'Intercept' (column 0)
    'key2' (column 1)

# Numerical data can be interprated as the categorical by C

In [9]:
y,X=patsy.dmatrices('v2~C(key2)',dt)
X

DesignMatrix with shape (8, 2)
  Intercept  C(key2)[T.1]
          1             0
          1             1
          1             0
          1             1
          1             0
          1             1
          1             0
          1             0
  Terms:
    'Intercept' (column 0)
    'C(key2)' (column 1)

In [10]:
dt['key2']=dt['key2'].map({0:'zero',1:'one'})

In [11]:
dt

,key1,key2,v1,v2
0,m,zero,1,-1.0
1,m,one,2,0.0
2,n,zero,3,2.5
3,n,one,4,-0.5
4,m,zero,5,4.0
5,n,one,6,-1.2
6,m,zero,7,0.2
7,n,zero,8,-1.7


In [12]:
y,X=patsy.dmatrices('v2~ key1 + key2',dt)

In [13]:
#here n=1,zero=1,means whose position is the greater as alfabetically ,that is 1
X

DesignMatrix with shape (8, 3)
  Intercept  key1[T.n]  key2[T.zero]
          1          0             1
          1          0             0
          1          1             1
          1          1             0
          1          0             1
          1          1             0
          1          0             1
          1          1             1
  Terms:
    'Intercept' (column 0)
    'key1' (column 1)
    'key2' (column 2)

In [14]:
y

DesignMatrix with shape (8, 1)
    v2
  -1.0
   0.0
   2.5
  -0.5
   4.0
  -1.2
   0.2
  -1.7
  Terms:
    'v2' (column 0)

In [15]:
y,X=patsy.dmatrices('v2~ key1 + key2 +key1:key2',dt)
X

DesignMatrix with shape (8, 4)
  Intercept  key1[T.n]  key2[T.zero]  key1[T.n]:key2[T.zero]
          1          0             1                       0
          1          0             0                       0
          1          1             1                       1
          1          1             0                       0
          1          0             1                       0
          1          1             0                       0
          1          0             1                       0
          1          1             1                       1
  Terms:
    'Intercept' (column 0)
    'key1' (column 1)
    'key2' (column 2)
    'key1:key2' (column 3)

In [17]:
#short hand notation for the above
y,X=patsy.dmatrices('v2~key1*key2',dt)
X

DesignMatrix with shape (8, 4)
  Intercept  key1[T.n]  key2[T.zero]  key1[T.n]:key2[T.zero]
          1          0             1                       0
          1          0             0                       0
          1          1             1                       1
          1          1             0                       0
          1          0             1                       0
          1          1             0                       0
          1          0             1                       0
          1          1             1                       1
  Terms:
    'Intercept' (column 0)
    'key1' (column 1)
    'key2' (column 2)
    'key1:key2' (column 3)

# Introduction to statmodel

In [1]:
import pandas as pd
import numpy as np
size=2
if isinstance(size,int):
    size=size,
    p= np.sqrt(4)*np.random.randn(*size)

In [2]:
p

array([-0.39349359, -0.83639284])

In [3]:
#the * unpack a tuple into multiple input argument.
def dnorm(mean, variance,size=1):
    if isinstance(size, int):
        size = size,
    return mean + np.sqrt(variance) * np.random.randn(*size)

In [4]:
def dnorm(mean, variance,size=1):
    if isinstance(size, int):
        size = size
    return mean + np.sqrt(variance) * np.random.randn(size)

In [5]:
np.random.seed(12345)
N = 100
X = np.c_[dnorm(0, 0.4,size=N),dnorm(0, 0.6,size=N),dnorm(0, 0.2,size=N)]

In [6]:
eps = dnorm(0, 0.1, size=N)

In [42]:
5+ 2*np.random.randn(2)

array([3.96112257, 3.88853939])

In [60]:
m=np.c_[1,2,3]
m

array([[1, 2, 3]], dtype=int32)

In [65]:
z=np.c_[np.array([1,2,3]),np.array([4,5,6])]
z

array([[1, 4],
       [2, 5],
       [3, 6]])

In [63]:
#translate the slice object to concatenating along the second axis
np.c_[np.array([1,2,3])]

array([[1],
       [2],
       [3]])

In [66]:
np.dot(z,[1,2])

array([ 9, 12, 15])

In [67]:
#isinstance() function is used to check whether the object or variable is an instance of the specified class type or datatype
isinstance('hey',str)

True

In [68]:
isinstance('hey',int)

False

In [22]:
#you can find the difference between numpy array and the list
[1,2]+[2,3]

[1, 2, 2, 3]

In [28]:
y=np.array([1,2,3])+np.array([4,5,6])
y

array([5, 7, 9])

In [30]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [1]:
def dnorm(mean,variance,size):
    if isinstance(size,int):
        size=size
    return mean+np.sqrt(variance)*np.random.randn(size)

In [9]:
import numpy as np
import pandas as pd
N=100
dnorm(0,0.4,size=N).shape

(100,)

In [8]:
dnorm(0,0.4,size=N).ndim

1

In [13]:
dnorm(0,0.4,size=N)

array([ 0.56243939, -0.13463953,  0.58628344, -0.34098478,  0.49349001,
        0.04170827, -0.9748685 ,  0.29042498,  0.92530603, -0.08143266,
       -0.27011351, -0.05047684, -0.27616263,  0.54372255,  0.458458  ,
       -0.83047552,  0.45477249, -0.42058899,  0.08790019,  0.57659807,
       -1.16477812,  1.75713293, -0.69062558,  0.60324567,  0.30711288,
        0.18863409,  0.71895227, -0.59767877,  0.86411873, -0.89998013,
        0.12656052,  0.36501484,  0.21703389, -0.03287003,  0.2976047 ,
       -0.49623153, -0.73420849, -0.85801535, -0.61681486, -1.18308255,
        1.38248059,  1.05084019, -0.59937455, -0.00546783, -0.25442377,
       -0.44513352, -1.08927756,  0.13166358,  0.41299544,  0.19982268,
       -0.29731462, -0.45822787,  0.29970359, -0.6532315 ,  1.12983156,
       -0.6506453 ,  0.37681154, -0.57108102,  0.77472068,  0.41443374,
        0.24459482, -0.83020523, -0.25756902,  0.92025002, -0.77536824,
       -0.28657149, -0.36793156,  0.13976535,  0.65211882,  1.20

In [10]:
X=np.c_[dnorm(0,0.4,size=N),dnorm(0,0.6,size=N),dnorm(0,0.2,size=N)]

In [12]:
X

array([[ 3.50806574e-02,  6.83451666e-02, -3.44619678e-01],
       [ 9.04122072e-01,  9.19089894e-02, -5.65863708e-01],
       [ 5.61492263e-01,  9.33308947e-02, -2.40700190e-02],
       [-3.20900843e-01, -5.10750912e-01,  4.92248219e-01],
       [-4.31708384e-01, -2.45231030e+00, -3.10548162e-01],
       [ 7.89959589e-01, -6.71616722e-01,  2.52506076e-01],
       [ 2.56088777e-01, -1.46047951e+00,  4.45328725e-01],
       [-7.48841169e-01, -6.82367436e-01, -3.10172524e-03],
       [-5.85149722e-02, -4.33583341e-01, -7.98632857e-02],
       [-5.29740636e-01, -5.35070490e-01,  1.19405993e+00],
       [-4.87569868e-01, -4.34851989e-02, -2.94907816e-02],
       [ 6.00442634e-01, -1.09945868e+00,  9.00482141e-01],
       [-9.16081393e-01,  4.14028604e-01,  3.09753262e-01],
       [ 1.62059040e+00, -9.46500024e-01, -2.57444010e-01],
       [ 8.78024707e-01,  1.08926816e+00,  2.27072273e-01],
       [-3.02444341e-01,  1.66124979e+00, -5.45426900e-01],
       [ 5.94080104e-01,  7.51474992e-01

In [15]:
X.shape

(100, 3)

In [16]:
X.ndim

2

In [37]:
eps=dnorm(0,0.1,size=N)
beta=[0.1,0.3,0.5]
y=np.dot(X,beta)+eps

In [38]:
y[:5]

array([ 0.74629763, -0.36141453, -0.02037057,  0.90237599, -0.98957383])

In [39]:
y.shape

(100,)

In [40]:
y.size

100

In [41]:
X_model=sm.add_constant(X)
X_model[:5]

array([[ 1.        ,  0.03508066,  0.06834517, -0.34461968],
       [ 1.        ,  0.90412207,  0.09190899, -0.56586371],
       [ 1.        ,  0.56149226,  0.09333089, -0.02407002],
       [ 1.        , -0.32090084, -0.51075091,  0.49224822],
       [ 1.        , -0.43170838, -2.4523103 , -0.31054816]])

In [42]:
model=sm.OLS(y,X)

In [45]:
results=model.fit()

In [47]:
results.params

array([0.16391079, 0.31560514, 0.41627092])

In [49]:
print(results.summary())

                                 OLS Regression Results                                
Dep. Variable:                      y   R-squared (uncentered):                   0.455
Model:                            OLS   Adj. R-squared (uncentered):              0.438
Method:                 Least Squares   F-statistic:                              27.01
Date:                Sat, 15 Aug 2020   Prob (F-statistic):                    8.79e-13
Time:                        08:10:07   Log-Likelihood:                         -34.078
No. Observations:                 100   AIC:                                      74.16
Df Residuals:                      97   BIC:                                      81.97
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

In [51]:
data=pd.DataFrame(X,columns=['col1','col2','col3'])
data.head(10)

,col1,col2,col3
0,0.035081,0.068345,-0.344620
1,0.904122,0.091909,-0.565864
2,0.561492,0.093331,-0.024070
3,-0.320901,-0.510751,0.492248
4,-0.431708,-2.452310,-0.310548
5,0.789960,-0.671617,0.252506
6,0.256089,-1.460480,0.445329
7,-0.748841,-0.682367,-0.003102
8,-0.058515,-0.433583,-0.079863
9,-0.529741,-0.535070,1.194060


In [52]:
results=smf.ols('y~col1+col2+col3',data).fit()

In [54]:
results.params

Intercept    0.011623
col1         0.163468
col2         0.317549
col3         0.417605
dtype: float64

In [56]:
results.tvalues

Intercept    0.331743
col1         3.051182
col2         6.777355
col3         5.028820
dtype: float64

In [57]:
results.predict(data[:5])

0   -0.104854
1   -0.047703
2    0.122995
3    0.002543
4   -0.967362
dtype: float64

# Estimating time series process

    Auto Regressive model: y(t) depent on the past value and the error term
    

    a time series is white noise if the variable are independent and identical distribution with a mean of zero.
    this means that all variables have the same variance and each value has zero corelation with other value in the series

    lag is the fixed amount of the passing time ;

In [8]:
b0=0.8
b1=-0.4
init_x=4
values=[init_x,init_x]
noise=dnorm(0,0.1,N)
for i in range(N):
    new_x=values[-1]*b0+values[-2]*b1+noise[i]
    values.append(new_x)

In [9]:
values

[4,
 4,
 1.3590859199528895,
 -0.6175977349271419,
 -1.5915271219179012,
 -0.8273088238525259,
 0.06409872307429197,
 -0.04095433456675324,
 -0.2167087425943065,
 -0.11851772419895801,
 0.5320886334341501,
 0.1660210100807381,
 -0.27586872869837575,
 -0.5169086760536276,
 0.08314191390920012,
 0.43256070224510634,
 -0.1260925227778245,
 -0.20911859235273084,
 0.07405048699980069,
 0.3154470915181632,
 0.2972018502207786,
 0.14694797382812738,
 -0.3970825961766872,
 -1.3091095721065074,
 -0.6877656763748996,
 0.0126931835827341,
 0.6956050784308603,
 0.017951605280529526,
 0.08066175153106547,
 0.7090567876754817,
 0.45869542121757884,
 -0.21640988014808515,
 -0.1427431226685951,
 -0.2342399056280288,
 -0.33656856028694926,
 -0.008328914092576606,
 0.2411084527739694,
 0.2558744934447782,
 0.3831899710409368,
 0.1881797255185512,
 -0.22926657708407727,
 -0.291338426487251,
 -0.5021915588018726,
 -0.4435774370823005,
 -0.27709322880825543,
 -0.19408830287022072,
 -0.003746125417631946,
 

In [11]:
len(values)

102

In [14]:
MAXLAG=5
import statsmodels.api as sm
model=sm.tsa.AR(values)
results=model.fit(MAXLAG)

In [15]:
results.params

array([ 0.00582631,  0.72192043, -0.40829336,  0.00075043,  0.11786968,
       -0.11515305])

# Introduction to scikit-learn

As an example,I am now using classic dataset from kaggle compitation about passenger survival rate on Titanic which sank in 1912. 

In [19]:
train=pd.read_csv(r'C:\Users\Shubhamay\Downloads\train_titanic.csv')
test=pd.read_csv(r'C:\Users\Shubhamay\Downloads\test_titanic.csv')
train[:5]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [20]:
train.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [21]:
test.isnull().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

In [23]:
impute_values=train['Age'].median()

In [25]:
train['Age']=train['Age'].fillna(impute_values)
test['Age']=test['Age'].fillna(impute_values)

In [26]:
train.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [29]:
train['isfemale']=(train['Sex']=='female').astype(int)

In [30]:
test['isfemale']=(test['Sex']=='female').astype(int)

In [32]:
predictor=['Pclass','isfemale','Age']

In [33]:
X_train=train[predictor].values
X_test=test[predictor].values

In [34]:
y_train=train['Survived'].values

In [35]:
X_train[:5]

array([[ 3.,  0., 22.],
       [ 1.,  1., 38.],
       [ 3.,  1., 26.],
       [ 1.,  1., 35.],
       [ 3.,  0., 35.]])

In [36]:
from sklearn.linear_model import LogisticRegression

In [40]:
lreg=LogisticRegression(solver='liblinear')

In [41]:
model=lreg.fit(X_train,y_train)

In [42]:
model

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='liblinear', tol=0.0001, verbose=0,
                   warm_start=False)

In [43]:
y_predict=lreg.predict(X_test)

In [45]:
y_predict[:10]

array([0, 0, 0, 0, 1, 0, 1, 0, 1, 0], dtype=int64)

In [58]:
from sklearn.model_selection import cross_val_score
scores=cross_val_score(model,X_train,y_train,cv=4)
scores

array([0.78026906, 0.81165919, 0.77130045, 0.78828829])

In [49]:
from sklearn.linear_model import LogisticRegressionCV

In [52]:
lreg_cv=LogisticRegressionCV(10)

In [60]:
model_cv=lreg_cv.fit(X_train,y_train)
model_cv

LogisticRegressionCV(Cs=10, class_weight=None, cv=None, dual=False,
                     fit_intercept=True, intercept_scaling=1.0, l1_ratios=None,
                     max_iter=100, multi_class='auto', n_jobs=None,
                     penalty='l2', random_state=None, refit=True, scoring=None,
                     solver='lbfgs', tol=0.0001, verbose=0)

In [61]:
cvscore=cross_val_score(model_cv,X_train,y_train,cv=4)

In [62]:
cvscore

array([0.71748879, 0.79820628, 0.77130045, 0.78828829])